In [3]:
import pandas as pd
import numpy as np
import transformers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Lee el archivo de Excel con dos columnas (texto y etiqueta)
df = pd.read_excel('cluster.xlsx')

# Separa las columnas de datos y etiquetas
df['Data'] = df['Data'].astype(str)
datos = df['Data'].tolist()
etiquetas = df['Etiqueta'].tolist()

# Preprocesamiento de datos
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
sequences = [tokenizer.encode(sent, add_special_tokens=True) for sent in datos]
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post', truncating='post', maxlen=100)

# Codifica las etiquetas de texto a números
le = LabelEncoder()
labels = le.fit_transform(etiquetas)
labels = np.asarray(labels)

# Separa los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Carga el modelo Transformer pre-entrenado y agrega la capa de clasificación
model = tf.keras.Sequential([
    transformers.TFAutoModel.from_pretrained('bert-base-uncased'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(60, activation='softmax')
])

# Compila el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

# Evalúa el modelo en el conjunto de prueba
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print("Precisión del modelo: {:.2f}%".format(acc * 100))

OSError: [Errno 28] No space left on device